In [ ]:
# ===============================================================
# 🧹 LỌC OUTLIER TRONG CỘT "Vol." CHO MÃ HPG + TẢI FILE VỀ MÁY
# ===============================================================

# --- 1. Import thư viện ---
import pandas as pd
from google.colab import files

# --- 2. Tải file lên từ máy (nếu chưa có trong Colab) ---
# Chạy dòng này nếu bạn chưa có file trong môi trường
# from google.colab import files
# uploaded = files.upload()

# --- 3. Đọc file dữ liệu gốc ---
df = pd.read_csv("merged_stock_data.csv")
print("✅ Số dòng ban đầu:", len(df))

# --- 4. Chuyển cột Vol. sang dạng số (xử lý 'K', 'M', ',') ---
def to_number(s):
    s = str(s)
    s = s.replace(',', '').upper().replace('K', 'E3').replace('M', 'E6')
    try:
        return float(s)
    except:
        return None

df['Vol.'] = df['Vol.'].map(to_number)

# --- 5. Lọc outlier của mã HPG bằng phương pháp IQR ---
hpg_vol = df[df['Ticker'] == 'HPG']['Vol.']
Q1 = hpg_vol.quantile(0.25)
Q3 = hpg_vol.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_no_outlier = df[~((df['Ticker'] == 'HPG') &
                     ((df['Vol.'] < lower_bound) | (df['Vol.'] > upper_bound)))]

print("✅ Số dòng sau khi lọc:", len(df_no_outlier))

# --- 6. Xuất file mới ---
output_file = "merged_stock_data_no_outlier_1.csv"
df_no_outlier.to_csv(output_file, index=False)
print(f"💾 File mới đã được lưu: {output_file}")

# --- 7. Tải file về máy ---
files.download(output_file)